
# Proyecto
### Erick Fernando López Fimbres.  
#### erick.lopez.fimbres@gmail.com

## Definición

Para definir que es el analisis semántico, primero debemos saber que es la semántica y la [RAE](http://dle.rae.es/?id=XVRDns5) lo define como:
*Disciplina que estudia el significado de las unidades lingüísticas y de sus combinaciones.*

Entonces el **análisis semántico** consiste en analizar el significado de un conjunto de palabras, símbolos y frases, en un contexto específico, con el fin de determinar el mensaje contenido en el texto.


## Modelado de tópicos
El proposito de este modelado es identificar tópicos a partir de una coleccion de documentos en nuestro caso llamado corpus a esta colección.

Existen diferentes aproximaciones para realizar el modelado de tópicos:
* PLSI (probabilistic latent semantic indexing - indexado probabilístico de semántica latente)
* NMF (non-negative matrix factorization - factorización de matrices no negativas)
* LDA (Latent Dirichlet Allocation - Asignación latente de Dirichlet).

## Identificaciń de tópicos
Basicamente el problema de identificación de topicos en un corpus se basa en un problema de reconocimiento de patrones.
Tenemos dos técnicas de clasificacion de acuerdo al grado de conocimiento disponible:

1. **Aprendizaje supervsado:** En este método las etiquetas de las clases ya estan identificadas con un cnjunto de datos ya clasificados con el cual es posible ajustar los modelos de la clase.  
2. **Aprendizaje no supervisado: ** En este método no hay una definición de las clases bien definidas ni que objetos pertenecen a cada clase. Una forma común de clasificar es el método de "clustering".



## Vectores de características

Un atributo es una propiedad simbólica o numérica de una propiedad de un objeto, que puede ser útil para clasificarlo en una determinada clase.

Se le denomina vector de características a la forma de sistematizar y homogenizar la descripción de elementos en un problema es utilizando el mismo conjunto de propiedades para todos ellos, sean útiles o no para la clasificación de un objeto en particular.

Una bolsa de palabras (o de tokens) es una colección de los términos considerados como "significativos" para describir los documentos en un corpus. La opción más simple y muy común es tomar el conjunto total de palabras utilizadas en el corpus, quizás eliminando solamente las palabras de paro.

El resultado típico de un método de clustering es un conjunto de vectores "prototipo" que sintetizan el modelo de cada una de las clases.

## Asignación latente de Dirichlet

La asignación latente de Dirichlet conocido como **LDA** es un método generativo para modelado de tópicos. El método asume que los documentos en un corpus se generan a partir de un conjunto de tópicos. Cada tópico, por otra parte, se representa como una distribución de frecuencias sobre un conjunto de términos/tokens.

## Distribución de probabilidad de Dirichlet

Una forma común de describir rasgos estadísticos de la ocurrencia de términos en un texto es mediante una función de distribución multinomial.

## Identificación de tópicos mediante LDA

Utilicemos el corpus limpio sobre Enfermedades generado en la fase de análisis léxico. Utilizaremos la descripción proporcionada en "nueva_descripcion" que está basada en el lexicón final.

In [2]:
from IPython.display import display
import pandas as pd
import numpy as np 
pd.options.display.max_colwidth = 150 

import nltk
import re
import json

from sklearn import decomposition
from sklearn.feature_extraction.text import CountVectorizer

In [5]:
file = 'datos/lexicon_enfermedades.json'
with open(file) as doc_enf:
    dict_enf = json.load(doc_enf)
enfDf = pd.DataFrame.from_dict(dict_enf)
enfDf = enfDf.reindex(columns = ["nombre", "descripcion", "nueva_descripcion","palabras_principales", "bigramas", 
                                        "bigramas_limpios", "todas_colocaciones"])
display(enfDf.head(5))

,nombre,descripcion,nueva_descripcion,palabras_principales,bigramas,bigramas_limpios,todas_colocaciones
0,Infantilismo,archivo windelfetischistjpg miniaturadeimagen un con infantilismo al tener el fetichismo del pañal el infantilismo es una enfermedad en la que per...,infantilismo archivo miniaturadeimagen infantilismo tener pañal infantilismo enfermedad persisten adulto_diversas_características anatomía_anatómi...,"[archivo, miniaturadeimagen, infantilismo, tener, pañal, infantilismo, enfermedad, persisten, adulto, diversas, características, anatomía, anatómi...","[[archivo, windelfetischistjpg], [windelfetischistjpg, miniaturadeimagen], [miniaturadeimagen, un], [un, con], [con, infantilismo], [infantilismo,...","[[adulto, diversas], [diversas, características], [características, anatomía], [anatomía, anatómicas], [anatómicas, fisiología], [fisiología, fisi...","[adulto diversas, diversas características, características anatomía, anatomía anatómicas, anatómicas fisiología, fisiología fisiológicas, psicolo..."
1,Pectus excavatum,el pectus excavatum es una deformidad congénita de la caja torácica caracterizada por pecho hundido en la región del esternón la probabilidad del ...,pectus_excavatum pectus excavatum deformidad_congénita caja_torácica caracterizada pecho_hundido región esternón probabilidad pectus_excavatum dep...,"[pectus, excavatum, deformidad, congénita, caja, torácica, caracterizada, pecho, hundido, región, esternón, probabilidad, pectus, excavatum, depen...","[[el, pectus], [pectus, excavatum], [excavatum, es], [es, una], [una, deformidad], [deformidad, congénita], [congénita, de], [de, la], [la, caja],...","[[pectus, excavatum], [deformidad, congénita], [caja, torácica], [torácica, caracterizada], [pecho, hundido], [pectus, excavatum], [excavatum, dep...","[pectus excavatum, deformidad congénita, caja torácica, torácica caracterizada, pecho hundido, pectus excavatum, excavatum depende, personas si, c..."
2,Pectus carinatum,el pectus carinatum es una deformidad de la caja torácica en la que el pecho protruye en quilla ave quilla de barco es menos frecuente que el pect...,pectus_carinatum pectus carinatum deformidad caja_torácica pecho ave menos_frecuente pectus_excavatum repercusiones_funcionales menos_importantes ...,"[pectus, carinatum, deformidad, caja, torácica, pecho, ave, menos, frecuente, pectus, excavatum, repercusiones, funcionales, menos, importantes, p...","[[el, pectus], [pectus, carinatum], [carinatum, es], [es, una], [una, deformidad], [deformidad, de], [de, la], [la, caja], [caja, torácica], [torá...","[[pectus, carinatum], [caja, torácica], [menos, frecuente], [pectus, excavatum], [repercusiones, funcionales], [menos, importantes], [importantes,...","[pectus carinatum, caja torácica, menos frecuente, pectus excavatum, repercusiones funcionales, menos importantes, importantes pues, pues tan, tan..."
3,Onfalocele,la onfalocele u onfalocele infantil también llamado exonfalocele se da en el momento en el que el embrión presenta las vísceras de la región abdom...,onfalocele onfalocele u_onfalocele infantil llamado da momento embrión_presenta vísceras región_abdominal cuerpo_aunque casi_siempre saco_datos on...,"[onfalocele, u, onfalocele, infantil, llamado, da, momento, embrión, presenta, vísceras, región, abdominal, cuerpo, aunque, casi, siempre, saco, d...","[[la, onfalocele], [onfalocele, u], [u, onfalocele], [onfalocele, infantil], [infantil, también], [también, llamado], [llamado, exonfalocele], [ex...","[[onfalocele, u], [u, onfalocele], [onfalocele, infantil], [embrión, presenta], [región, abdominal], [cuerpo, aunque], [aunque, casi], [casi, siem...","[onfalocele u, u onfalocele, onfalocele infantil, embrión presenta, región abdominal, cuerpo aunque, aunque casi, casi siempre, saco datos, onfalo..."
4,Período prepatogénico,el período prepatogénico precede a las manifestaciones clínica s y está conformado por las condiciones del huésped el agente y el medio ambiente e...,período_prepatogénico pe

Un objeto CountVectorizer convierte una colección de documentos en una matriz de conteo de tokens. Esta clase contiene métodos para limpiar los datos, generar bigramas y otras funciones útiles que ya realizamos en la fase de análisis léxico. El resultado es el siguiente:

In [19]:
# Crear el contador
vectorizer = CountVectorizer()
# Construir la matriz documentos-términos
X = vectorizer.fit_transform(enfDf.nueva_descripcion)
X_array = X.toarray()
X_vocab = np.array(vectorizer.get_feature_names())

print("""Se generó una matriz de tamaño {}\n
Forma de los vectores (un fragmento): \n{}"""
      .format(X.shape, X_array[2, 100:200], X_vocab[0:100]))

Se generó una matriz de tamaño (1632, 87901)

Forma de los vectores (un fragmento): 
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [20]:
tokens = []
for i, x in zip(range(len(X_vocab)), X_array[2]):
    if x >1 :
        tokens.append([i, X_vocab[i], x])

df = pd.DataFrame(tokens, columns = ["Índice", "Token", "Frecuencia"])
print("Muestra del vocabulario utilizado")
display(df)

Muestra del vocabulario utilizado


,Índice,Token,Frecuencia
0,60673,pectus_carinatum,2
1,60674,pectus_excavatum,2


El módulo [sklearn.decomposition](http://scikit-learn.org/stable/modules/classes.html#module-sklearn.decomposition) proporciona diversos métodos de descomposición de matrices, entre ellos los métodos **LatentDirichletAllocation** y **NMF**. Generamos un objeto que realice la transformación de la matriz de documentos-terminos, siendo n_topics el número de nuevos atributos ($k$), en este caso, el número de tópicos que queremos identificar.

In [21]:
num_topics = 10
# Creación del operador. En la versión 0.19, el argumento n_topics cambia a n_components
lda = decomposition.LatentDirichletAllocation(n_topics=num_topics,
                                              learning_method='online')
# Factorización de la matriz documentos-términos
lda_topics = lda.fit_transform(X)

print("Tamaño de la matriz documentos-términos:", X.shape)
print("Tamaño de la matriz de distribución documentos-tópicos:", lda_topics.shape)
print("Tamaño de la matriz de distribución tópicos-términos:", lda.components_.shape)

/home/erickmon/anaconda3/lib/python3.5/site-packages/sklearn/decomposition/online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


Tamaño de la matriz documentos-términos: (1632, 87901)
Tamaño de la matriz de distribución documentos-tópicos: (1632, 10)
Tamaño de la matriz de distribución tópicos-términos: (10, 87901)


In [22]:
# Atributos originales a mostrar en cada vector de características, 
# ordenados por importancia       
num_top_words = 50
lda_topic_words = []
for topic in lda.components_:
    word_idx = np.argsort(topic)[::-1][0:num_top_words]
    lda_topic_words.append([X_vocab[i] for i in word_idx])

for i, row in zip(range(len(lda_topic_words)), lda_topic_words):
    print(i, row, "\n")

0 ['cáncer', 'enfermedad', 'pacientes', 'tratamiento', 'casos', 'síntomas', 'paciente', 'personas', 'síndrome', 'si', 'diagnóstico', 'embargo', 'virus', 'presencia', 'gluten', 'riesgo', 'debido', 'infección', 'células', 'uso', 'niños', 'forma', 'aunque', 'pulmón', 'años', 'después', 'estudio', 'dolor', 'así', 'vih', 'salud', 'aparición', 'mayoría', 'sangre', 'enfermedades', 'menos', 'próstata', 'través', 'desarrollo', 'estudios', 'puede_ser', 'además', 'parte', 'general', 'ejemplo', 'caso', 'causa', 'tumor', 'aumento', 'posible'] 

1 ['baron_cohen', 'color_brightblue', 'shift_text_bar', 'from_till', 'esófago', 'complejo_carney', 'diésel', 'humo_diésel', 'materia_particulada', 'carginógenos', 'fragile_x', 'barrett', 'grupo_iarc', 'síndrome_x_frágil', 'whipple', 'carcinogenicidad', 'contaminación_atmosférica', 'fmr', 'pth', 'autobuses', 'enfermedades_cardiovasculares', 'teratoma', 'fregoli', 'reflujo_laringofaríngeo', 'gen_fmr', 'surfactante', 'acento', 'pedatricacom', 'película', 'diese

In [28]:
# Normalizar la matriz de coeficientes
lda_topics_norm = lda_topics / np.sum(lda_topics, axis=1, keepdims=True) 

# Obtener los títulos de los documentos
page_titles = np.asarray(list(enfDf.nombre))

num_groups = len(set(page_titles))
lda_topics_grouped = np.zeros((num_groups, num_topics))
for i, name in enumerate(sorted(set(page_titles))):
    # Promedios, útiles cuando hay documentos con títulos repetidos
    lda_topics_grouped[i, :] = np.mean(lda_topics_norm[page_titles == name, :], axis=0)

lda_pages = pd.DataFrame(data=lda_topics_grouped, index=page_titles, 
                   columns=["T" + str(i) for i in range(num_topics)])

display(lda_pages[0:20])
display(lda_pages[500:520])
display(lda_pages[1000:1020])

,T0,T1,T2,T3,T4,T5,T6,T7,T8,T9
Infantilismo,0.064631,0.000602,0.000602,0.000602,0.000602,0.000602,0.000602,0.000602,0.361140,0.570012
Pectus excavatum,0.000807,0.000806,0.000806,0.000806,0.000806,0.000806,0.000806,0.000806,0.000806,0.992742
Pectus carinatum,0.184457,0.000714,0.000714,0.038031,0.000714,0.602694,0.000714,0.000714,0.000714,0.170532
Onfalocele,0.430361,0.000962,0.000962,0.351305,0.000962,0.000962,0.000962,0.000962,0.000962,0.211603
Período prepatogénico,0.668828,0.000376,0.000376,0.262034,0.000376,0.000376,0.000376,0.000376,0.000376,0.066506
Enfermedad gastrointestinal,0.245099,0.000129,0.000129,0.000129,0.000129,0.000129,0.000129,0.000129,0.000129,0.753873
Mal de montaña crónico,0.000962,0.000962,0.000962,0.379235,0.000962,0.000962,0.000962,0.000962,0.000962,0.613073
Fibrosis,0.000981,0.000980,0.000980,0.000980,0.000980,0.000980,0.000980,0.000980,0.000980,0.991176
Meteoropatía,0.774936,0.001408,0.001408,0.001408,0.182226,0.001408,0.001408,0.001408,0.001408,0.032979
Linfangitis,0.000953,0.000952,0.000952,0.000952,0.000952,0.000952,0.000952,0.000952,0.000952,0.991428


,T0,T1,T2,T3,T4,T5,T6,T7,T8,T9
Glucogenosis tipo III,0.999452,0.000061,0.000061,0.000061,0.000061,0.000061,0.000061,0.000061,0.000061,0.000061
Síndrome de Larsen,0.563322,0.000529,0.000529,0.000529,0.000529,0.295007,0.000529,0.000529,0.000529,0.137968
Criptosporidiosis,0.821830,0.000094,0.000094,0.000094,0.000094,0.000094,0.000094,0.000094,0.000094,0.177420
Síndrome poliglandular autoinmune,0.637205,0.233915,0.000224,0.000224,0.000224,0.000224,0.000224,0.000224,0.000224,0.127314
Botulismo,0.578190,0.417643,0.000521,0.000521,0.000521,0.000521,0.000521,0.000521,0.000521,0.000521
Adiposis dolorosa,0.003449,0.003448,0.565937,0.003448,0.003448,0.003448,0.003448,0.003448,0.003448,0.406476
Neuralgia del glosofaríngeo,0.804181,0.000099,0.000099,0.000099,0.000099,0.000099,0.000099,0.000099,0.000099,0.195031
Síndrome de Young,0.001299,0.001299,0.001299,0.001299,0.001299,0.001299,0.270584,0.001299,0.001299,0.719026
Insomnio familiar fatal,0.452756,0.000302,0.000302,0.000302,0.000302,0.000302,0.000302,0.405052,0.000302,0.140077
Síndrome de Muenke,0.001283,0.001282,0.001282,0.001282,0.001282,0.001282,0.001282,0.001282,0.001282,0.988461


,T0,T1,T2,T3,T4,T5,T6,T7,T8,T9
Síndrome de Felty,0.512483,0.482074,0.000680,0.000680,0.000680,0.000680,0.000680,0.000680,0.000680,0.000680
Enfermedad de Still,0.932863,0.000149,0.000149,0.000149,0.000149,0.000149,0.000149,0.000149,0.000149,0.065949
Síndrome de Couvade,0.001177,0.001176,0.001176,0.367255,0.001176,0.001176,0.001176,0.001177,0.001176,0.623333
Síndrome de Netter,0.246650,0.002564,0.002564,0.002564,0.462122,0.002564,0.002564,0.002564,0.002564,0.273279
Síndrome de Mirizzi,0.000918,0.000917,0.000917,0.000917,0.000917,0.000917,0.000917,0.000917,0.000917,0.991743
Síndrome de Meigs,0.596871,0.001786,0.001786,0.351734,0.001786,0.038893,0.001786,0.001786,0.001786,0.001786
Enfermedad de Köhler,0.338542,0.000400,0.000400,0.000400,0.000400,0.000400,0.378099,0.000400,0.000400,0.280559
Trastorno de somatización,0.285898,0.000585,0.000585,0.000585,0.000585,0.000585,0.268875,0.000585,0.000585,0.441133
Síndrome de Ackerman,0.260230,0.000408,0.000408,0.000408,0.000408,0.000408,0.000408,0.000408,0.363358,0.373555
Hemorragia de Duret,0.353931,0.000296,0.000296,0.000296,0.000296,0.000296,0.328862,0.000296,0.000296,0.315136


Para ver un poco mejor el como es la pertenencia de topicos utilizaremos la tecnica de dendogramas para el agrupamiento de tópicos por palabras y documentos de la siguiente forma:

In [ ]:
from matplotlib import pyplot as plt
from scipy.cluster.hierarchy import dendrogram, linkage
%matplotlib inline

Z_lda = linkage(lda.components_, 'complete')
plt.figure(figsize=(12, 8))
dendrogram(Z_lda, leaf_font_size=14)
plt.show()

/home/erickmon/anaconda3/lib/python3.5/site-packages/matplotlib/colors.py:680: MatplotlibDeprecationWarning: The is_string_like function was deprecated in version 2.1.
  not cbook.is_string_like(colors[0]):
